# Inference
This notebook is intended to depict the results of inference for the xView2 Dataset. 

In [1]:
from utils.inference_step import inference
from utils.helperfunctions import load_checkpoint, find_best_checkpoint, get_data_folder
from utils.dataset import xView2Dataset, collate_fn_test, image_transform
from pathlib import Path
from torch.utils.data import DataLoader
import os
import yaml
from model.siameseNetwork import SiameseUnet
from model.uNet import UNet_ResNet50
from model.loss import FocalLoss, combined_loss_function
import torch
base_dir = Path(os.getcwd()).parent  # Gehe einen Ordner zurück vom aktuellen Arbeitsverzeichnis
config_path = base_dir / "notebooks" / "00_config.yaml"
print(base_dir)

with open(config_path, "r") as file:
    config = yaml.safe_load(file)



/dss/dsshome1/08/di97ren/04-geo-oma24/xView2SiameseUNet


## Get all pathes and create Dataset and DataLoader

In [2]:

DATA_ROOT, TEST_ROOT, VAL_IMG, TEST_LABEL, TEST_TARGET, TEST_PNG_IMAGES = get_data_folder("test", main_dataset = False)

test_dataset = xView2Dataset(png_path=TEST_PNG_IMAGES,
image_transform = image_transform(), inference = True)

test_dataloader = DataLoader(
    test_dataset, batch_size = 32,
    collate_fn = collate_fn_test,
    shuffle = False,
    num_workers = 2
)

In [3]:
USER = config["data"]["user"]
#USER_PATH = Path(f"/dss/dsstbyfs02/pn49ci/pn49ci-dss-0022/users/{USER}")
USER_HOME_PATH = Path(f"/dss/dsshome1/08/{USER}")

# Pathes to store experiment informations in:
EXPERIMENT_GROUP = config["data"]["experiment_group"]
EXPERIMENT_ID = config["data"]["experiment_id"]
EXPERIMENT_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "tensorboard_logs" / EXPERIMENT_ID
EXPERIMENT_DIR.mkdir(parents=True, exist_ok=True)

print(EXPERIMENT_DIR)

# Auch Checkpoints-Verzeichnis erstellen
CHECKPOINTS_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "checkpoints"
CHECKPOINTS_DIR.mkdir(parents=True, exist_ok=True)

# Logfiles-Verzeichnis erstellen
LOGFILES_DIR = USER_HOME_PATH / EXPERIMENT_GROUP / "logfiles" / EXPERIMENT_ID
LOGFILES_DIR.mkdir(parents=True, exist_ok=True)
print(f"Logfiles werden gespeichert in: {LOGFILES_DIR}")


/dss/dsshome1/08/di97ren/xView2_all_data/tensorboard_logs/001
Logfiles werden gespeichert in: /dss/dsshome1/08/di97ren/xView2_all_data/logfiles/001


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"


# Modell initialisieren
model = SiameseUnet(num_pre_classes=2, num_post_classes=6)
model.to(device)

best_checkpoint_path = find_best_checkpoint(CHECKPOINTS_DIR, EXPERIMENT_ID)
# Besten Checkpoint laden
model = load_checkpoint(model, best_checkpoint_path)
results = inference(model, test_dataloader)


Loaded raw state_dict from /dss/dsshome1/08/di97ren/xView2_all_data/checkpoints/001_best_siamese_unet_state.pth
Checkpoint erfolgreich in DataParallel-Modell geladen.


: 

: 

# Plot the results:

In [ ]:
from utils.viz import vizualize_predictions

In [ ]:
vizualize_predictions(results)